<a href="https://colab.research.google.com/github/CarolineLaure/One_Qubit_TensorFlow_example/blob/master/Copy_of_Caroline_Onequbit_Optimal_Control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import time
print(tf.__version__)

2.2.0-rc1


In [3]:
#Deleseleuc parameters

delta1=2*np.pi*560*10**6
sigma_total=2*np.pi*13*10**3
delta2=-delta1+sigma_total
Omega1=2*np.pi*60*10**6
Omega2=2*np.pi*36*10**6

class Propagator:
    def __init__(self, no_of_steps, delta_t, dim):
        
        self.dim=3
        self.delta_t = delta_t
        self.x1 = tf.constant(
            [[0, 1, 0], [1, 0, 0], [0, 0, 0]], dtype=tf.complex128
        )
        self.x2 = tf.constant(
            [[0 , 0, 0], [0, 0, 1], [1, 0, 0]], dtype=tf.complex128
        )

        self.x3 = tf.constant(
            [[0 , 0, 0], [0, 1, 0], [0, 0, 1]], dtype=tf.complex128
        )

        self.x4 = tf.constant(
            [[0 , 0, 0], [0, 0, 0], [0, 0, 1]], dtype=tf.complex128
        )

        self.generators = tf.stack([self.x3, self.x4, self.x1, self.x2])   
        self.ctrl_amplitudes = tf.Variable(
            tf.zeros([no_of_steps, 4], dtype=tf.float64), dtype=tf.float64
        )

        """
            self.contraction_array determines the neccessity for the extra
            matrix multiplication step in the recursive method self.propagate()
            when the intermediate computation array has length not divisible
            by 2
        """
        self.contraction_array = []
        contraction_array_length = int(np.floor(np.log2(no_of_steps)))
        temp_no_of_steps = no_of_steps
        for i in range(contraction_array_length):
            self.contraction_array.append(bool(np.mod(temp_no_of_steps, 2)))
            temp_no_of_steps = np.floor(temp_no_of_steps / 2)
  
    """
        exponentials() computes a vector matrix exponential after multiplying
        each self.ctrl_amplitudes row with a the vector of matrices in
        self.generators
    """
    def exponentials(self):
        regularized_amplitudes = 1 / np.sqrt(2) * tf.math.tanh(
            self.ctrl_amplitudes
        )

        exponents = tf.linalg.tensordot(
            tf.cast(regularized_amplitudes, dtype=tf.complex128),
            -2 * np.pi *(0 + 1j) * self.delta_t * self.generators, 1
        )
        return tf.linalg.expm(exponents)
    
    """
        propagate  computes the final propagator by recursively multiplying
        each odd element in the list of matrices with each even element --
        if the length of the array is not divisible by 2 an extra computation
        step is added
    """
    def propagate(self):
        step_exps = self.exponentials()
        for is_odd in self.contraction_array:
            if is_odd:
                odd_exp = step_exps[-1, :, :]
                step_exps = tf.linalg.matmul(
                    step_exps[1::2, :, :], step_exps[0:-1:2, :, :]
                )
                step_exps = tf.concat([
                    step_exps[0:-1, :, :],
                    [tf.linalg.matmul(odd_exp, step_exps[-1, :, :])]
                ], 0)
            else:
                step_exps = tf.linalg.matmul(
                    step_exps[1::2, :, :], step_exps[0::2, :, :]
                )
        return tf.squeeze(step_exps)

    """
        __call__ computes the final propagator fidelity squared with the
        identity operator
    """
    
    @tf.function
    def infidelity(self):
        propagator = self.propagate()
        tr = tf.linalg.trace(tf.linalg.matmul(self.x1, propagator))
        return 1 - tf.math.real(tr * tf.math.conj(tr)) / (3 ** 2)

propagator = Propagator(1000, 10**(-3), 3)

optimizer = tf.keras.optimizers.Adam(0.01)

propagator.ctrl_amplitudes.assign(
    tf.random.uniform([1000, 4], -1, 1, dtype=tf.float64)
)

propagator.propagate()
def optimization_step():
    with tf.GradientTape() as tape:
        infidelity = propagator.infidelity()
    gradients = tape.gradient(infidelity, [propagator.ctrl_amplitudes])
    optimizer.apply_gradients(zip(gradients, [propagator.ctrl_amplitudes]))
    return infidelity

steps = range(100)
for step in steps:
    current_infidelity = optimization_step()
    print('step %2d: infidelity=%2.5f' %
          (step, current_infidelity))
    
propagator.ctrl_amplitudes.numpy()

#propagator.delta_t
#propagator.no_of_steps

step  0: infidelity=0.99561
step  1: infidelity=0.99218
step  2: infidelity=0.98779
step  3: infidelity=0.98246
step  4: infidelity=0.97618
step  5: infidelity=0.96898
step  6: infidelity=0.96086
step  7: infidelity=0.95184
step  8: infidelity=0.94194
step  9: infidelity=0.93118
step 10: infidelity=0.91961
step 11: infidelity=0.90727
step 12: infidelity=0.89419
step 13: infidelity=0.88044
step 14: infidelity=0.86607
step 15: infidelity=0.85116
step 16: infidelity=0.83577
step 17: infidelity=0.81999
step 18: infidelity=0.80392
step 19: infidelity=0.78765
step 20: infidelity=0.77129
step 21: infidelity=0.75494
step 22: infidelity=0.73868
step 23: infidelity=0.72264
step 24: infidelity=0.70689
step 25: infidelity=0.69153
step 26: infidelity=0.67666
step 27: infidelity=0.66234
step 28: infidelity=0.64863
step 29: infidelity=0.63559
step 30: infidelity=0.62325
step 31: infidelity=0.61164
step 32: infidelity=0.60076
step 33: infidelity=0.59060
step 34: infidelity=0.58113
step 35: infidelity=

array([[-0.22455473, -0.91328591, -0.85046095, -1.45705482],
       [ 0.73718653, -1.19997224, -0.3385713 , -1.99058393],
       [-0.67132053, -0.69511244,  0.52968821, -0.52795224],
       ...,
       [ 0.68061291,  1.35740162, -0.33539012,  0.94341818],
       [-0.63204594, -0.08249646, -0.55103008,  1.78972465],
       [-0.82603978, -0.55418365,  0.52130804,  2.02974554]])

My Hamiltonian is: -\delta|e><e| - sigma_total |r><r| - (Omega1/2)(|g><e| + |e><g|) - (Omega2/2)(|e><r| + |r><e|)